In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train['Sex_binario'] = train['Sex'].map({'male':0, 'female':1})

In [4]:
test['Sex_binario'] = test['Sex'].map({'male':0, 'female':1})

In [5]:
# selecione as colunas que são numericos, menos passengerid, survived
X = train.select_dtypes(include=np.number).drop(['PassengerId', 'Survived'], axis=1).fillna(0)
y = train['Survived']

In [6]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_binario
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0


In [7]:
X

,Pclass,Age,SibSp,Parch,Fare,Sex_binario
0,3,22.0,1,0,7.2500,0
1,1,38.0,1,0,71.2833,1
2,3,26.0,0,0,7.9250,1
3,1,35.0,1,0,53.1000,1
4,3,35.0,0,0,8.0500,0
...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0
887,1,19.0,0,0,30.0000,1
888,3,0.0,1,2,23.4500,1
889,1,26.0,0,0,30.0000,0


In [8]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.pipeline import make_pipeline

In [12]:
kf = KFold(n_splits=2, random_state=0,shuffle=True)

In [13]:
# criação de matriz de zeros, com o formatado de numeros de linhas com 4 colunas (numero do modelo)
second_level = np.zeros((X.shape[0], 4))

In [14]:
second_level

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [15]:
# tr , ts numeros das linhas 
for tr, ts in kf.split(X, y):
    Xtr, Xval = X.iloc[tr], X.iloc[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]

    print(f'treino {tr}')
    print(f'teste {ts}')
    
    print(f'TAMANHO {len(tr)}')
    print(f'TAMANHO {len(ts)}')

treino [  3   4   7   9  11  13  16  19  23  24  25  26  28  29  32  33  36  41
  42  43  44  47  53  56  57  58  59  61  63  67  69  70  72  73  80  82
  83  84  86  87  88  91  93  94  95  98  99 106 110 111 112 114 115 117
 119 120 121 123 125 126 128 129 130 131 134 135 136 137 138 139 143 146
 147 148 149 151 152 153 160 163 164 166 168 169 174 176 177 180 182 183
 184 186 189 191 192 195 197 199 201 203 207 209 212 213 216 217 219 221
 224 225 226 227 228 232 234 237 244 246 248 254 256 257 259 260 265 269
 273 274 275 276 277 280 281 282 284 286 287 288 289 290 291 292 295 296
 297 300 305 307 314 321 322 323 324 325 326 328 329 330 332 335 336 339
 340 341 343 347 348 349 353 355 357 359 360 365 368 369 370 371 373 376
 377 381 383 385 387 388 393 394 396 398 400 404 405 407 410 411 414 415
 417 419 421 423 426 427 429 430 431 433 435 437 438 442 444 445 446 448
 449 450 454 455 459 461 463 464 466 468 469 470 472 473 478 480 486 488
 490 497 498 499 502 504 507 508 509 510 512

In [16]:
for tr, ts in kf.split(X, y):
    Xtr, Xval = X.iloc[tr], X.iloc[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]
    
    # n_jobs = 6 nucleos
    rf  = RandomForestClassifier(n_estimators=100, n_jobs=6, random_state=10)
    rf.fit(Xtr, ytr)
    # probabilidade de um eveneto pertencer a classe 1 ou a classe 0
    prf = rf.predict_proba(Xval)[:,1]
    prf_ = (prf > 0.5).astype(int)
    
    print(f'RF Acurácia: {accuracy_score(yval, prf_)} - Log Loss: {log_loss(yval, prf)}')    
    
    et = ExtraTreesClassifier(n_estimators=100, n_jobs=6, random_state=10)
    et.fit(Xtr, ytr)
    pet = et.predict_proba(Xval)[:,1]
    pet_ = (pet > 0.5).astype(int)
    
    print(f'ET Acurácia: {accuracy_score(yval, pet_)} - Log Loss: {log_loss(yval, pet)}')
    
    lr1 = make_pipeline(StandardScaler(), LogisticRegression())
    lr1.fit(Xtr, ytr)
    plr1 = lr1.predict_proba(Xval)[:,1]
    plr1_1 = (plr1 > 0.5).astype(int)
    
    print(f'LR StdScaler Acurácia: {accuracy_score(yval, plr1_1)} - Log Loss: {log_loss(yval, plr1)}')
        
    lr2 = make_pipeline(MinMaxScaler(), LogisticRegression())
    lr2.fit(Xtr, ytr)
    plr2 = lr2.predict_proba(Xval)[:,1]
    plr2_ = (plr2 > 0.5).astype(int)
    
    print(f'LR MinMax Acurácia: {accuracy_score(yval, plr2_)} - Log Loss: {log_loss(yval, plr2)} \n')
    
    # matriz com as previsões
    
    
    second_level[ts, 0] = prf
    second_level[ts, 1] = pet
    second_level[ts, 2] = plr1
    second_level[ts, 3] = plr2
    

RF Acurácia: 0.7869955156950673 - Log Loss: 0.8891877224352214
ET Acurácia: 0.7802690582959642 - Log Loss: 1.7277767849421113
LR StdScaler Acurácia: 0.7713004484304933 - Log Loss: 0.4548870233895985
LR MinMax Acurácia: 0.773542600896861 - Log Loss: 0.45730271249069515 

     Pclass   Age  SibSp  Parch     Fare  Sex_binario
3         1  35.0      1      0  53.1000            1
4         3  35.0      0      0   8.0500            0
7         3   2.0      3      1  21.0750            0
9         2  14.0      1      0  30.0708            1
11        1  58.0      0      0  26.5500            1
..      ...   ...    ...    ...      ...          ...
873       3  47.0      0      0   9.0000            0
874       2  28.0      1      0  24.0000            1
879       1  56.0      0      1  83.1583            1
884       3  25.0      0      0   7.0500            0
888       3   0.0      1      2  23.4500            1

[445 rows x 6 columns]
     Pclass   Age  SibSp  Parch     Fare  Sex_binario
0  

In [ ]:
for tr, ts in kf.split(X,y):
    Xtr, Xval = second_level[tr], second_level[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]
    
    lr_stack = LogisticRegression(C=1.)
    lr_stack.fit(Xtr, ytr)
    plr_stack = lr_stack.predict_proba(Xval)[:,1]
    plr_stack_ = (plr_stack > 0.5).astype(int)
    
    print(f'Stack Acurácia: {accuracy_score(yval, plr_stack_)} - Log Loss: {log_loss(yval, plr_stack)} \n')

In [ ]:
pd.DataFrame(np.corrcoef(second_level.T))

In [ ]:

second_level